In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU: Tesla T4


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


path = "/content/drive/MyDrive/Data_Image"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Image_dataset = datasets.ImageFolder(root=path, transform=transform)


In [ ]:
# Chia dữ liệu thành tập train và tập test
train_dataset, test_dataset = random_split(Image_dataset, [0.7, 0.3])

batch_size = 32
# Tạo DataLoader để nạp dữ liệu theo batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(Image_dataset)

1346

In [ ]:
from torchvision.models import vgg16
import torch.nn as nn

class Vison_model(nn.Module):
    def __init__(self, num_classes=16):
        super(Vison_model, self).__init__()

        self.vgg16 = vgg16(pretrained=False).to(device)

        n_inputs = self.vgg16.classifier[6].in_features

        self.vgg16.classifier[6] = nn.Sequential(
            nn.Linear(n_inputs, num_classes, bias=True),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.vgg16(x)
        return x


num_classes = len(Image_dataset.classes)
vision_model = Vison_model(num_classes).to(device)

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):
        # Chuyển dữ liệu và mô hình lên GPU
        X, y = X.to(device), y.to(device)
        model.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for (X, y) in data_loader:
            # Chuyển dữ liệu lên GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y).item()
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(vision_model.parameters(), lr=0.0001)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()  # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc


In [ ]:
torch.manual_seed(42)

epochs = 30
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    vision_model.train()

    train_step(data_loader=train_loader,
        model=vision_model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )

    vision_model.eval()
    test_step(data_loader=test_loader,
        model=vision_model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

Epoch: 0
---------
Train loss: 2.65320 | Train accuracy: 13.12%
Test loss: 2.53989 | Test accuracy: 14.20%

Epoch: 1
---------
Train loss: 2.55237 | Train accuracy: 17.81%
Test loss: 2.39138 | Test accuracy: 17.64%

Epoch: 2
---------
Train loss: 2.33356 | Train accuracy: 25.71%
Test loss: 2.35874 | Test accuracy: 28.30%

Epoch: 3
---------
Train loss: 2.13124 | Train accuracy: 32.76%
Test loss: 1.99791 | Test accuracy: 38.17%

Epoch: 4
---------
Train loss: 1.82219 | Train accuracy: 44.20%
Test loss: 1.86127 | Test accuracy: 41.21%

Epoch: 5
---------
Train loss: 1.58884 | Train accuracy: 51.94%
Test loss: 1.71222 | Test accuracy: 47.81%

Epoch: 6
---------
Train loss: 1.39126 | Train accuracy: 56.33%
Test loss: 1.66275 | Test accuracy: 47.70%

Epoch: 7
---------
Train loss: 1.09020 | Train accuracy: 66.19%
Test loss: 1.55452 | Test accuracy: 52.53%

Epoch: 8
---------
Train loss: 0.86264 | Train accuracy: 71.67%
Test loss: 1.61536 | Test accuracy: 54.14%

Epoch: 9
---------
Train los